In [1]:

    
    importalas()

    #behívás
    read()
    nevekkiszurese(df_szkivonat)
    kozlemenykiszures(df_szkivonat)
    betutipus()

    #kategorizálás
    besorolas_kd_taborok()
    nevkategoriak()
    kurzusok_ber()
    nav()
    tamogatasok()
    osztondijak()
    epulet()
    maradek()
    
    #összegzés
    #összegzés()
    
    #dash
    
FM_progprojekt()

NameError: name 'importalas' is not defined

# Import

In [56]:
def importalas():
    import pandas as pd
    import math
    import numpy as np
    import xlrd
    import matplotlib.pyplot as plt
    import plotly.graph_objects as go

    
importalas()

# 1.) Rajk alapítvány CIB számlakivonatának beolvasása 

In [33]:
df_szkivonat=pd.read_excel('osszes.xlsx', skiprows=8,sep=';', header=0) 
df_szkivonat=df_szkivonat.drop(columns='Számlaegyenleg')
df_szkivonat
df_szkivonat=df_szkivonat.dropna()

# Utalók neveinek kiszűrése

In [34]:
def nevkiszures(df):
    nevek=[]
    df['Közleménylista']=df['Leírás'].str.split()
    for i, row in df.iterrows():
# Bejövő utalások
        if df.loc[i,'Közleménylista'][0]=='Bankon':
            kozlemenylista=df.loc[i,'Közleménylista'][6]+' '+df.loc[i,'Közleménylista'][7]
            nevek.append(kozlemenylista)
        elif df.loc[i,'Közleménylista'][0]=='Bejövő':
            kozlemenylista=df.loc[i,'Közleménylista'][5]+' '+df.loc[i,'Közleménylista'][6] 
            nevek.append(kozlemenylista)
        elif df.loc[i,'Közleménylista'][0]=="7DE":
            kozlemenylista=df.loc[i,'Közleménylista'][4]+' '+df.loc[i,'Közleménylista'][5] 
            nevek.append(kozlemenylista)
        elif df.loc[i,'Közleménylista'][0]=="VIBER":
            kozlemenylista=df.loc[i,'Közleménylista'][8][2:]+' '+df.loc[i,'Közleménylista'][9] 
            nevek.append(kozlemenylista)
# Kimenő utalások
        elif df.loc[i,'Közleménylista'][0]=='Kimenő':
            index_vege = df.loc[i,'Közleménylista'].index('Közlemény:')
            kozlemenylista=" ".join(df.loc[i,'Közleménylista'][5:(index_vege)])
            nevek.append(kozlemenylista)
            
        elif 'BETÉT' in df.loc[i,'Közleménylista']:
            kozlemenylista='Lekötött betét'
            nevek.append(kozlemenylista)
            
# Bankköltség
        elif df.loc[i,'Közleménylista'][0]=="JUTALÉK":
            kozlemenylista="Bankköltség" 
            nevek.append(kozlemenylista)
# Készpénzfelvétel kártyával
        elif df.loc[i,'Közleménylista'][0]=="KÁRTYA":
            kozlemenylista="Készpénzfelvétel" 
            nevek.append(kozlemenylista)

# Kártyaterhelés 
        elif df.loc[i,'Közleménylista'][0]=="BANKKÁRTYA" and df.loc[i,'Közleménylista'][1]=="TERHELÉS" :       
            if 'USD' in df.loc[i,'Közleménylista']:
                index = df.loc[i,'Közleménylista'].index('USD')
                index_vege = df.loc[i,'Közleménylista'].index('Értéknap:')
                kozlemenylista=" ".join(df.loc[i,'Közleménylista'][(index+6):(index_vege-1)])
                nevek.append(kozlemenylista)
            elif 'EUR' in df.loc[i,'Közleménylista'] and 'HUF' not in df.loc[i,'Közleménylista']:
                index = df.loc[i,'Közleménylista'].index('EUR')
                index_vege = df.loc[i,'Közleménylista'].index('Értéknap:')
                kozlemenylista=" ".join(df.loc[i,'Közleménylista'][(index+6):(index_vege-1)])
                nevek.append(kozlemenylista)
            
            elif ('EUR' and 'HUF') in df.loc[i,'Közleménylista']:
                index = df.loc[i,'Közleménylista'].index('HUF')
                index_vege = df.loc[i,'Közleménylista'].index('Értéknap:')
                kozlemenylista=" ".join(df.loc[i,'Közleménylista'][(index+7):(index_vege-2)])
                nevek.append(kozlemenylista)

            elif 'HUF' in df.loc[i,'Közleménylista']:
                index = df.loc[i,'Közleménylista'].index('HUF')
                index_vege = df.loc[i,'Közleménylista'].index('Értéknap:')
                kozlemenylista=" ".join(df.loc[i,'Közleménylista'][(index+6):(index_vege-1)])
                nevek.append(kozlemenylista)
        else: 
            kozlemenylista='Egyéb'
            nevek.append(kozlemenylista)
    
    df['Nevek']=nevek


nevkiszures(df_szkivonat)

# Közlemények kiszűrése (Amennyiben írtak közleményt)

In [35]:
#Közlemények kiszűrése

def kozlemenykiszures(df):
    kozlemenyek=[]
    for j, col in df.iterrows():
        eleje=0
        for k in range(len(df.loc[j,'Közleménylista'])):
            vege=0
            if df.loc[j,'Közleménylista'][k]=='Közlemény:':
                while df.loc[j,'Közleménylista'][k+1]!='Értéknap:':
                    kozlemenyek.append(df.loc[j,'Közleménylista'][k+1])
                    eleje+=1  
                    vege+=1
                    k+=1
                if vege!=0:
                    kozlemenyek.append('-----------------------')
        if eleje==0:
            kozlemenyek.append('Nincs_közlemény')
            kozlemenyek.append('-----------------------')
    return kozlemenyek

kozlemenyek=kozlemenykiszures(df_szkivonat)
kozlemenyek

['KD',
 '-----------------------',
 'SZUHAI',
 'FLÓRA',
 '-',
 'KOLLÉGIUMI',
 'DÍJ',
 '-----------------------',
 'havi',
 'rendszeres',
 'támogatás',
 '(Hajósi',
 'Péter,',
 'Girbicz',
 'Nikolett)',
 '-----------------------',
 'KD',
 'SMarci',
 '12',
 '+',
 'bünti',
 '500',
 '-----------------------',
 'KÉGL',
 'VIRÁG',
 'KD',
 'JANUÁR',
 '-----------------------',
 'Kolidij',
 '-',
 'Január',
 '-----------------------',
 'Nincs_közlemény',
 '-----------------------',
 '2078/16510',
 '-----------------------',
 'Nincs_közlemény',
 '-----------------------',
 'Tagdíj',
 'néhány',
 'év',
 '-----------------------',
 'MUKODESI',
 'TAMOGATAS',
 '-----------------------',
 'NyT',
 '-----------------------',
 'Herbák',
 'Mihály',
 'KD',
 '+',
 'késés',
 '-----------------------',
 'Barát',
 'Andor',
 'Kornél',
 'KD',
 '+',
 'késés',
 '-----------------------',
 'mikrotámogatás',
 '-----------------------',
 'X.',
 'XI.',
 'hónap',
 '-----------------------',
 'Nincs_közlemény',
 '---------

In [36]:
def osszefuzes():    
#Közlemények mergelése a df-el
    merge=' '.join(str(v) for v in kozlemenyek)
    közlemény=merge.split('-----------------------')
    df_kozlemeny=pd.DataFrame(közlemény)
    df_kozlemeny=df_kozlemeny.drop(index=len(df_kozlemeny)-1)
    df_kozlemeny=df_kozlemeny.set_index(df_szkivonat.index)
    df_szkivonat['Közlemény_lista']=df_kozlemeny

df_utalasok=df_szkivonat.drop(columns=['Közleménylista','Leírás'])


osszefuzes()

In [37]:
pd.options.display.max_rows = 100

#Betűtípus egységesítése
def nagybetu(inputom):
    inputom=inputom.split(' ')
    for i in range(len(inputom)):
        inputom[i]=inputom[i].casefold().capitalize()
    return ' '.join(inputom)

def kisbetu(kozlemeny):
    kozlemeny=kozlemeny.casefold()
    return kozlemeny

def betutipus():
  
    #Nevek a kategorizáláshoz
    df_utalasok['Nevek'] = df_utalasok['Nevek'].apply(nagybetu)
    df_utalasok["Nevek_lista"] = df_utalasok["Nevek"].str.split()


    #Közlemény átalakítása kisbetűs listává + splittelés több típusú karakternél
    df_utalasok["Közlemény"] = df_szkivonat["Közlemény_lista"].apply(kisbetu)
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace('_', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace('-', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace('-', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace('.', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace(',', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace('+', ' ')
    df_utalasok["Közlemény"] = df_utalasok["Közlemény"].str.replace(':', ' ')
    df_utalasok["Közlemény_lista"] = df_utalasok["Közlemény"].str.split()
    
betutipus()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd]
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]"
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ..."
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]"
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]"
...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]"
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]"
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]"
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]"


# Tételek besorolása kategóriákba

In [38]:
pd.options.display.max_rows = 100

def besorolas_kd_taborok():

    for i, row in df_utalasok.iterrows():

        df_utalasok.loc[i,'Kategória']="Besorolásra vár"

    # kollégiumi díj és táborok bevételeinek kiszűrése

        kd=['kd','díj','koll','kollégiumi','kolidíj','koli','dij', 'kolidij', 'tartozas', 'kolidíj,','kol.dij',
            'kol.díj','hónap', 'havi', 'bünti', 'bunt','bünt']
        taborok=['nyt', 'nyári','tt','téli', 'tábordíj', 'tábor', 'tt,','teli', 'tabor', 'tt-díj','télitábor']

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in kd) and df_utalasok.loc[i,'Összeg']>0 :
            df_utalasok.loc[i,'Kategória']="Kollégiumi díj"

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in taborok) and df_utalasok.loc[i,'Összeg']>0 :
            df_utalasok.loc[i,'Kategória']="Táborok"

besorolas_kd_taborok()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista,Kategória
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd],Kollégiumi díj
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]",Kollégiumi díj
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ...",Kollégiumi díj
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]",Kollégiumi díj
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]",Kollégiumi díj
...,...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]",Besorolásra vár
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]",Kollégiumi díj
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]",Táborok
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]",Kollégiumi díj


In [39]:
# Sok random dolog kiszűrése az utalók nevéből

pd.options.display.max_rows = 100

def nevkategoriak():
    for i, row in df_utalasok.iterrows():

        category_dict_nevek = {'Bankköltség': 'Bankköltség', 'Budapesti Corvinus Egyetem': 'BCE', 
                         'Készpénzfelvétel': 'Készpénzes tételek', 'Közép-európai Egyetem' : 'CEU', 
                        'Central Médiacsoport' : 'Támogatás', 'Facebk' : 'Jeszkes költések ', 'Temesvári Kft' : 'Táborok', 
                              'Lekötött Betét' : 'Betét lekötés'}
        for key in category_dict_nevek:
            if df_utalasok.loc[i,'Nevek']== key:
                df_utalasok.loc[i,'Kategória']= category_dict_nevek[key]
            
nevkategoriak()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista,Kategória
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd],Kollégiumi díj
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]",Kollégiumi díj
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ...",Kollégiumi díj
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]",Kollégiumi díj
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]",Kollégiumi díj
...,...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]",Jeszkes költések
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]",Kollégiumi díj
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]",Táborok
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]",Kollégiumi díj


In [40]:
# Kurzusok és munkabérek kiszűrése a közleményból

pd.options.display.max_rows = 100

def kurzusok_ber():
    for i, row in df_utalasok.iterrows():

        kurzusok=['megbízási']
        munkabér=['munkabér','költségtérítés']

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in kurzusok):
            df_utalasok.loc[i,'Kategória']="Kurzusok"

    # ha vállalkozó a kurzustartó akko 200k és sajnos közleményben nincs értelmezhető pattern, ezért szürők erre
        if df_utalasok.loc[i,'Kategória']!='Készpénzfelvétel' and df_utalasok.loc[i,'Összeg']==-200000 :
            df_utalasok.loc[i,'Kategória']="Kurzusok"

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in munkabér):
            df_utalasok.loc[i,'Kategória']="Bérek"

        if df_utalasok.loc[i,'Nevek']=='Főző Zsolt' and df_utalasok.loc[i,'Összeg']<0 :
            df_utalasok.loc[i,'Kategória']="Bérek"

        if df_utalasok.loc[i,'Nevek']=='Szombati Enikő E.v.' and df_utalasok.loc[i,'Összeg']<0 :
            df_utalasok.loc[i,'Kategória']="Bérek"

            
kurzusok_ber()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista,Kategória
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd],Kollégiumi díj
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]",Kollégiumi díj
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ...",Kollégiumi díj
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]",Kollégiumi díj
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]",Kollégiumi díj
...,...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]",Jeszkes költések
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]",Kollégiumi díj
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]",Táborok
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]",Kollégiumi díj


In [41]:
# Nav munkabérek utáni járulékbefizetés vs 1% visszatérités

pd.options.display.max_rows = 100

def nav():
    
    for i, row in df_utalasok.iterrows():

        if 'Nav' in df_utalasok.loc[i,'Nevek_lista'] and df_utalasok.loc[i,'Összeg']<0 :
            df_utalasok.loc[i,'Kategória']= 'Nav befizetések'

        elif 'Nav' in df_utalasok.loc[i,'Nevek_lista'] and df_utalasok.loc[i,'Összeg']>0 :
            df_utalasok.loc[i,'Kategória']= 'Adó 1%'
        
nav()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista,Kategória
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd],Kollégiumi díj
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]",Kollégiumi díj
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ...",Kollégiumi díj
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]",Kollégiumi díj
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]",Kollégiumi díj
...,...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]",Jeszkes költések
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]",Kollégiumi díj
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]",Táborok
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]",Kollégiumi díj


In [42]:
#Támogatások kiszűrése

pd.options.display.max_rows = 100

def tamogatasok():
    for i, row in df_utalasok.iterrows():


        tamogatas=['szerződés', 'tamogatas', 'támogatás', 'mikrotámogatás', 'rendszeres', 'kurzusok', 'rafi', 'alap', 
                   'adomány','voszk', 'tagdíj','loranth',' Mozer','rajkos','szerz','támogatási']

        tamogatas_nevek=['Otp Mobil']

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in tamogatas):
            df_utalasok.loc[i,'Kategória']="Támogatás"

        if any(név in df_utalasok.loc[i,'Nevek'] for név in tamogatas_nevek):
            df_utalasok.loc[i,'Kategória']="Támogatás"
        
# tanulmányi ösztöndíj
def osztondijak():
    
    osztondij=['ösztöndíj','odüsszeusz']
    for i, row in df_utalasok.iterrows():

        if any(közl in df_utalasok.loc[i,'Közlemény_lista'] for közl in osztondij):
            df_utalasok.loc[i,'Kategória']="Tanulmányi ösztöndíj"
        
        
# Épület
def epulet():
    for i, row in df_utalasok.iterrows():

        epulet=['Delta', 'Criterion','Higiénia','FAB', 'Hosting']

        if any(név in df_utalasok.loc[i,'Nevek_lista'] for név in epulet):
            df_utalasok.loc[i,'Kategória']="Épület"
            
tamogatasok()
osztondijak()
epulet()
df_utalasok

,Könyvelési dátum,Összeg,Nevek,Nevek_lista,Közlemény,Közlemény_lista,Kategória
0,2020-01-02,10700.0,Biró Marcell,"[Biró, Marcell]",kd,[kd],Kollégiumi díj
1,2020-01-02,22500.0,Szuhai Flóra,"[Szuhai, Flóra]",szuhai flóra kollégiumi díj,"[szuhai, flóra, kollégiumi, díj]",Kollégiumi díj
2,2020-01-02,8000.0,Hajósi Péter,"[Hajósi, Péter]",havi rendszeres támogatás (hajósi péter girb...,"[havi, rendszeres, támogatás, (hajósi, péter, ...",Támogatás
3,2020-01-02,23000.0,Őrlősy Dorottya,"[Őrlősy, Dorottya]",kd smarci 12 bünti 500,"[kd, smarci, 12, bünti, 500]",Kollégiumi díj
4,2020-01-02,5700.0,Kégl Virág,"[Kégl, Virág]",kégl virág kd január,"[kégl, virág, kd, január]",Kollégiumi díj
...,...,...,...,...,...,...,...
858,2020-05-18,-25000.0,Facebk,[Facebk],nincs közlemény,"[nincs, közlemény]",Jeszkes költések
859,2020-05-19,61900.0,Gyenes Ana,"[Gyenes, Ana]",kd gymárk 02 03 04 05 bünti,"[kd, gymárk, 02, 03, 04, 05, bünti]",Kollégiumi díj
860,2020-05-19,7000.0,Gyenes Ana,"[Gyenes, Ana]",tt díj gyenes márk,"[tt, díj, gyenes, márk]",Táborok
861,2020-05-19,19200.0,Lázár Ádám,"[Lázár, Ádám]",3 4 5 bünti,"[3, 4, 5, bünti]",Kollégiumi díj


In [43]:
#Maradék nem besorolt tételek felcimkézése.
pd.options.display.max_rows = 100
def maradek():
    for i, row in df_utalasok.iterrows():
        if df_utalasok.loc[i,'Kategória']=='Besorolásra vár' and df_utalasok.loc[i,'Összeg']<0 :
            df_utalasok.loc[i,'Kategória']='Egyéb költések'
        if df_utalasok.loc[i,'Kategória']=='Besorolásra vár' and df_utalasok.loc[i,'Összeg']>0 :
            df_utalasok.loc[i,'Kategória']='Támogatás'
            
maradek()

Final_dataframe=df_utalasok.drop(columns=['Közlemény_lista','Nevek_lista'])
Final_dataframe['Könyvelési dátum'] = pd.to_datetime(Final_dataframe['Könyvelési dátum'])

# Összegzés

In [99]:
pd.options.display.max_rows = 10


összegzés=Final_dataframe.groupby(['Könyvelési dátum','Kategória'])['Összeg'].sum().reset_index().sort_values(by='Könyvelési dátum')
összegzés['Hónap']=összegzés['Könyvelési dátum'].dt.strftime('%Y-%m')
összegzés['Év']=összegzés['Könyvelési dátum'].dt.strftime('%Y')    
    
összegzés

,Könyvelési dátum,Kategória,Összeg,Hónap,Év
0,2020-01-02,Kollégiumi díj,81900.00,2020-01,2020
1,2020-01-02,Támogatás,8000.00,2020-01,2020
2,2020-01-03,Bankköltség,-204.00,2020-01,2020
3,2020-01-03,Egyéb költések,-5080.00,2020-01,2020
4,2020-01-03,Támogatás,20000.00,2020-01,2020
...,...,...,...,...,...
307,2020-05-18,Egyéb költések,-50897.14,2020-05,2020
310,2020-05-18,Támogatás,6947.60,2020-05,2020
312,2020-05-19,Kollégiumi díj,81100.00,2020-05,2020
311,2020-05-19,Bankköltség,-21.00,2020-05,2020


In [100]:
havi=összegzés.groupby(['Hónap'])['Összeg'].sum().reset_index()
havi_kat=összegzés.groupby(['Hónap', 'Kategória'])['Összeg'].sum().reset_index() 
éves_kat=összegzés.groupby(['Év', 'Kategória'])['Összeg'].sum().reset_index()

In [101]:
#Kiadás, Bevétel, Egyenleg dataframek létrehozása kategória szerinti havi bontásban

#bevételek
összegzés_bev=összegzés.loc[összegzés['Összeg'] > 0]
bev = összegzés_bev.groupby(['Hónap', 'Kategória'])['Összeg'].sum().reset_index()
bev = bev.pivot(index='Hónap', columns='Kategória', values='Összeg')
bev = bev.fillna(0)
bev = bev.reset_index()

#Kiadások
összegzés_kiad=összegzés.loc[összegzés['Összeg'] < 0]
kiad = összegzés_kiad.groupby(['Hónap', 'Kategória'])['Összeg'].sum().reset_index()
kiad = kiad.pivot(index='Hónap', columns='Kategória', values='Összeg')
kiad = kiad.reset_index()

#Egyenleg
egyenleg=összegzés.groupby(['Hónap', 'Kategória'])['Összeg'].sum().reset_index() 
egyenleg=egyenleg.pivot(index='Hónap', columns='Kategória', values='Összeg')
egyenleg=egyenleg.fillna(0)
egyenleg=egyenleg.reset_index()

In [107]:
import plotly.express as px

def px_stacked_bar(df, color_name='Kategória', y_name='Összeg', **pxargs):
    idx_col = df.index.name
    m = pd.melt(df.reset_index(), id_vars=idx_col, var_name=color_name, value_name=y_name)
    return px.bar(m, x=idx_col, y=y_name, color=color_name, **pxargs)

In [103]:
px_stacked_bar(egyenleg.set_index('Hónap'))

# Dash app

In [108]:
from dash import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import os
import flask
from dash_core_components import Dropdown, Graph
from dash_html_components import Div, H3, Link
from dash.dependencies import Input, Output, State
from plotly import graph_objs as go
from datetime import datetime as dt
import plotly.express as px


In [ ]:
fig = px_stacked_bar(egyenleg.set_index('Hónap'))

app = dash.Dash()

app.title = 'Rajk CF'

app.layout = Div(
    [
        H3(
            children="Rajk CF kimutatás",
            style={
                "text-align": "center",
                "width": "60%",
                "margin": "0 auto",
                "padding": "20px 20px",
                "font-family": "Times New Roman",
            },
        ),

        dcc.RadioItems(
                        id = 'Tipus', 
                        options=[
                            {'label': 'Bevételek', 'value': 'bev'},
                            {'label': 'Kiadások', 'value': 'kiad'},
                            {'label': 'Egyenleg', 'value': 'egyenleg'},
                                
                            ],  
                            value = 'egyenleg' 
                        ),  
        dcc.Graph(id="Graph", figure=fig),
    ]
)




@app.callback(
    Output('Graph', 'figure'),
    [Input('Tipus', 'value')])

def update_figure(Tipus):
    
    if Tipus == 'bev':
        fig = px_stacked_bar(bev.set_index('Hónap'))
    
    elif Tipus == 'kiad':
        fig = px_stacked_bar(kiad.set_index('Hónap'))
        
    elif Tipus == 'egyenleg':
        fig = px_stacked_bar(egyenleg.set_index('Hónap'))
    
    return fig


if __name__ == "__main__":
    app.run_server(debug=False, host="rajk.uni-corvinus.hu", port=8045)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://rajk.uni-corvinus.hu:8045/ (Press CTRL+C to quit)
